# csv_时间_枚举输出解析器

## csv输出解析器

当您想要返回以逗号分隔的项目列表时，可以使用此输出解析器。

In [4]:
from langchain_openai import ChatOpenAI
from langchain_core.prompts import PromptTemplate
from langchain_core.output_parsers import CommaSeparatedListOutputParser
from dotenv import load_dotenv
import os

load_dotenv()

api_key = os.getenv("ZHIPUAI_API_KEY")
base_url = os.getenv("ZHIPUAI_API_BASE")

chat = ChatOpenAI(api_key=api_key, base_url=base_url, temperature=0.3, max_tokens=8192, model="glm-4-plus")

output_parser = CommaSeparatedListOutputParser()

format_instructions = "你的响应格式是csv格式以逗号作为分隔符的列表，例如:item1,item2,item3"
prompt = PromptTemplate(
    template="{format_instructions}\n请列出{subject}",
    input_variables=["subject"],
    partial_variables={"format_instructions": format_instructions},
)

chain = prompt | chat | output_parser

In [5]:
format_instructions = output_parser.get_format_instructions()
format_instructions

'Your response should be a list of comma separated values, eg: `foo, bar, baz` or `foo,bar,baz`'

In [6]:
print(chain.invoke({"subject":"请列出10个水果"}))

['苹果', '香蕉', '橙子', '草莓', '葡萄', '西瓜', '梨', '芒果', '桃子', '菠萝']


In [7]:
print(chain.invoke({"subject":"请列出5个冰激凌口味"}))

['香草', '巧克力', '草莓', '抹茶', '芒果']


## 日期时间解析器
此 OutputParser 可用于将 LLM 输出解析为日期时间格式。

In [18]:
from langchain.output_parsers import DatetimeOutputParser
from langchain_core.prompts import PromptTemplate

output_parser = DatetimeOutputParser()

format_instructions = output_parser.get_format_instructions()
format_instructions = "你的响应格式必须且只能返回日期时间字符串，格式为：YYYY-MM-DDThh:mm:ss.sssZ。 不要包含任何其他解释文字。 "
new_prompt = PromptTemplate(
    template="{format_instructions}\n请列出{subject}",
    input_variables=["subject"],
    partial_variables={"format_instructions": format_instructions},
)

chain = new_prompt | chat | output_parser

In [19]:
print(chain.invoke({"subject": "比特币是什么时间创立的？"}))

2009-01-03 18:15:05


In [20]:
print(chain.invoke({"subject": "中国是什么时间决定开始发展上海自贸区的？"}))

2013-07-03 00:00:00


In [21]:
print(chain.invoke({"subject": "美国是什么时间开始新冠疫情封城的？"}))

2020-03-16 23:59:00


## Enum parser 枚举解析器

In [36]:
from langchain.output_parsers.enum import EnumOutputParser
from langchain_core.prompts import PromptTemplate
from enum import Enum

class Fruit(Enum):
    APPLE = "苹果"
    BANANA = "香蕉"
    ORANGE = "橙子"
    PEACH = "桃子"
    WATERMELON = "西瓜"

output_parser = EnumOutputParser(enum=Fruit)

format_instructions = output_parser.get_format_instructions()
format_instructions = "你的响应请从以下选项选择之一：苹果、香蕉、橙子、桃子、西瓜。不要有其他选项回答。"

new_template = """
{person}的喜欢吃什么水果？
{format_instructions}
"""
new_prompt = PromptTemplate(
    template=new_template,
    input_variables=["person"],
    partial_variables={"format_instructions": format_instructions},
)

chain = new_prompt | chat | output_parser


In [37]:
chain.invoke({"person": "亚洲人"})

<Fruit.WATERMELON: '西瓜'>

In [38]:
chain.invoke({"person": "欧洲人"})

<Fruit.APPLE: '苹果'>

In [41]:
chain.invoke({"person": "美洲人"})

<Fruit.BANANA: '香蕉'>